In [ ]:
import pint.models as model
import pint.toa as toa
import pint.logging
import pint.fitter
from pint.modelutils import model_ecliptic_to_equatorial
import numpy as np
import astropy.units as u
import pint.derived_quantities as dq

pint.logging.setup(level="ERROR")

**J1221-0633 eclipsing calculations**

For J1221-0633, we see additional delays around superior conjuntion at 820 MHz, probably due to additional electron content around the companion that the signal must travel through when the pulsar eclipses (no signal at 350 MHz). First I want to figure out the DM bump corresponding to this delay (roughly 200 microseconds).

In [ ]:
def SC_dDM(dt,freq):
    """ Calculate additional column density at superior conjunction (given additional timing delay)
    
    Parameters
    ==========
    dt: quantity, additional dispersive timing delay due to plasma around companion
    freq: quantity, observing frequency of measured delay
    
    Returns
    =======
    dDM_sup: quantity, additional column density at superior conjunction (cm^-2)
    """
    Dconst = 4.148808e3*(1e6*u.Hz)**2*u.cm**3*u.s*u.pc**-1 # HBOPA, eqn. 4.6
    dDM_sup = (dt*freq**2/Dconst).to(u.pc*u.cm**-3) # extra DM corresponding to delay
    return dDM_sup.to(u.cm**-2)

To compare to a similar calculation in Freire (2005; ASPCS, p. 407), I convert DM (column density) to 1/cm^2. The result is comparable to the column density of J2051-0827 (Stappers et al. 2001).

In [ ]:
dDM_1221 = SC_dDM(200.0*1e-6*u.s,820.0*(1e6*u.Hz))
dDM_1221

How big might the companion be based on the duration of the eclipse? Assume inclination angle of 90 deg for now, and consider an eclipse that lasts some fraction of an orbit (based on gap between ingress-egress).

In [ ]:
def Rplasma(a, frac_eclipse):
    """ Estimate companion (plasma?) radius based on extent of eclipse
    
    Parameters
    ==========
    a: quantity, separation between the pulsar and companion
    frac_eclipse: float, fraction of the orbit where pulsar signal is obscured
    
    Returns
    =======
    Rplasma: quantity, companion's plasma(?) radius (Solar Radii)
    """
    print(f"Nominal pulsar-companion separation: {a.value:.4f} Rsun")
    ing2sup = frac_eclipse*180.0*u.deg # angle from ingress to superior conjunction
    Rplasma = a*np.sin(ing2sup)
    print(f"Companion's estimated plasma radius: {Rplasma.value:.4f} Rsun")
    return Rplasma

eclipsing_psrs = ["J1317-0157","J1221-0633"]
frac_eclipses = [0.15,0.16]
rpDict = {}

for ep,fe in zip(eclipsing_psrs,frac_eclipses):
    par_path = f"data/{ep}_swiggum+22.par"
    tim_path = f"data/{ep}_swiggum+22.tim"
    mo = model.get_model(par_path)
    to = toa.get_TOAs(tim_path,model=mo)
    fo = pint.fitter.WLSFitter(to,mo)
    x = fo.fit_toas()
    a = mo['A1'].quantity.to(u.R_sun) # assume 90 deg inclination
    print(f"{ep} is obscured for {fe:.0%} of an orbit...")
    rp = Rplasma(a,fe)
    rpDict[mo['PSR'].value]=rp
    if ep != eclipsing_psrs[-1]: print()

For J1221, I can use the estimated companion's plasma radius to get some idea of the electron density near the companion (using the additional column density around superior conjunction). This is proabbly ~order of magnitude.

In [ ]:
(dDM_1221/rpDict['J1221-0633']).to(u.cm**-3)

Next, I want to use the equation from Eggleton et al. (1983) to estimate the size of the companion's Roche Lobe, and thus, the path length the pulsar might be encountering around superior conjunction. Do this for a few possible inclination angles and assume pulsar mass is 1.4 solar masses.

In [ ]:
def RL_Eggleton(a,mc,mp=1.4*u.solMass):
    """ Calculate Roche Lobe size in solar radii (see Eggleton 1983)
    
    Parameters
    ==========
    a: quantity, separation between the pulsar and companion
    mc: quantity, companion mass (solar masses)
    mp: pulsar mass (default: 1.4 Msun), optional
    
    Returns
    =======
    R_L: quantity, Roche Lobe size (solar radii)
    """
    q = mc/mp
    R_L = 0.49*a*q**(2.0/3)/(0.6*q**(2.0/3)+np.log(1.0+q**(1.0/3)))
    return R_L.to(u.R_sun)

eclipsing_psrs = ["J1317-0157","J1221-0633"]
incs = [90.0,80.0,70.0,60.0] # trial inclination angles, degrees

for ep in eclipsing_psrs:
    par_path = f"data/{ep}_swiggum+22.par"
    tim_path = f"data/{ep}_swiggum+22.tim"
    mo = model.get_model(par_path)
    to = toa.get_TOAs(tim_path,model=mo)
    fo = pint.fitter.WLSFitter(to,mo)
    x = fo.fit_toas()
    print(f"Calculations for {ep}...")
    
    for inc in incs:
        a = (mo['A1'].quantity * np.sin(inc*u.deg)).to(u.R_sun)
        mp = 1.4*u.M_sun
        mc = dq.companion_mass(mo['PB'].quantity, mo['A1'].quantity, i=inc*u.deg, mp=mp)
        R_L = RL_Eggleton(a,mc,mp=mp)
        RlRp_ratio = R_L/rpDict[ep]
        
        mc_str = f"{mc.value:.4f}"
        q_str = f"{mc/mp:.4f}"
        a_str = f"{a.value:.4f}"
        rl_str = f"{R_L.value:.5f}"
        rlrp_str = f"{RlRp_ratio:.2f}"
        
        print(f"  i = {inc} deg; mc = {mc_str} Msun; q = {q_str}; a = {a_str} Rsun; RL = {rl_str} Rsun; RL/Rp = {rlrp_str}")
        
    if ep != eclipsing_psrs[-1]: print()

Therefore (as in Sec. 2.1 of Freire 2005), the Roche Lobe (R_L above) is much smaller than the size of the plasma cloud (from rpDict), and matter responsible for increased dispersive delays is not bound to the companion (i.e. companion is losing mass). This is true for both J1221 and J1317.